Importamos las librerias necesarias para operar.

In [1]:
import pandas as pd
import gzip
import zipfile

In [3]:
# Leer el DataFrame df_games desde un archivo CSV
df_games = pd.read_csv('/DataSets/df_games_id.csv')

# Leer el DataFrame df_user desde un archivo CSV dentro de un archivo ZIP
with zipfile.ZipFile('/DataSets/df_user_id.zip', 'r') as zipf:
    with zipf.open('df_user_id.csv') as csv_file:
        df_user = pd.read_csv(csv_file)

# Leer el DataFrame df_reviews desde un archivo CSV
df_reviews = pd.read_csv('/DataSets/df_user_reviews.csv')

In [4]:
df_user

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,30,Day of Defeat,7
2,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,3830,Psychonauts,333
...,...,...,...,...
3285244,76561198329548331,304930,Unturned,677
3285245,76561198329548331,227940,Heroes & Generals,43
3285246,76561198329548331,388490,One Way To Die: Steam Edition,3
3285247,76561198329548331,521570,You Have 10 Seconds 2,4


## Año con más horas jugadas por genero

In [8]:
def PlayTimeGenre(genre):
    # Paso 1: Filtrar juegos por género
    filtered_games = df_games[df_games['genres'].str.contains(genre, case=False, na=False)]

    # Paso 2: Unir DataFrames
    merged_df = pd.merge(df_user, filtered_games, left_on='item_id', right_on='id')

    # Paso 3: Agregar horas de juego por año
    grouped_df = merged_df.groupby('release_date')['playtime_forever'].sum().reset_index()

    # Paso 4: Identificar el año con más horas jugadas
    max_year = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['release_date']
    
    # Extraer solo el año
    year_only = max_year.split('-')[0]

    result_string = 'El año con más horas jugadas para el género ' + genre + ' es: ' + str(year_only)
    return result_string

# Ejemplo de uso
result = PlayTimeGenre('Action')
print(result)

El año con más horas jugadas para el género Action es: 2012


## Usuario con mas horas jugadas por genero.

In [9]:
def UserForGenre(genre):
    # Paso 1: Filtrar juegos por género
    filtered_games = df_games[df_games['genres'].str.contains(genre, case=False, na=False)]

    # Paso 2: Unir DataFrames
    merged_df = pd.merge(df_user, filtered_games, left_on='item_id', right_on='id')

    # Paso 3: Convertir la columna 'release_date' a formato de fecha y extraer el año
    merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], errors='coerce')
    merged_df = merged_df.dropna(subset=['release_date'])  # Eliminar filas con fechas nulas

    merged_df['release_year'] = merged_df['release_date'].dt.year

    # Paso 4: Agregar horas de juego por año y usuario
    grouped_df = merged_df.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()

    # Paso 5: Identificar el usuario con más horas jugadas
    max_user = grouped_df.loc[grouped_df['playtime_forever'].idxmax()]['user_id']

    # Paso 6: Obtener la lista de acumulación de horas jugadas por año para ese usuario
    user_hours_by_year = grouped_df[grouped_df['user_id'] == max_user][['release_year', 'playtime_forever']]

    user_hours_list = [{"Año": int(year), "Horas": hours} for year, hours in user_hours_by_year.values]

    return {"Usuario con más horas jugadas para Género " + genre: max_user, "Horas jugadas": user_hours_list}

# Ejemplo de uso
result = UserForGenre('Action')
print(result)

{'Usuario con más horas jugadas para Género Action': 'Evilutional', 'Horas jugadas': [{'Año': 2009, 'Horas': 2037}, {'Año': 2010, 'Horas': 4102}, {'Año': 2011, 'Horas': 1968}, {'Año': 2012, 'Horas': 680991}, {'Año': 2013, 'Horas': 69726}, {'Año': 2014, 'Horas': 24226}, {'Año': 2015, 'Horas': 112}, {'Año': 2016, 'Horas': 1291}, {'Año': 2017, 'Horas': 10894}]}


## Top tres más recomendadas.

In [10]:
def UsersRecommend(year):
    # Convertir la columna 'release_date' a tipo datetime
    df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

    # Filtrar las reviews para el año dado con recomendación negativa y comentarios negativos
    filtered_reviews = df_reviews[
        (df_reviews['recommend'] == True) &
        (df_reviews['sentiment_analiysis'] != 0) &  
        (pd.to_datetime(df_games['release_date']).dt.year == year)
]
    # Unir con el DataFrame de juegos para obtener los nombres de los juegos
    merged_reviews = pd.merge(filtered_reviews, df_games[['id', 'app_name']], left_on='item_id', right_on='id')

    # Contar la cantidad de veces que aparece cada juego
    top_games = merged_reviews['app_name'].value_counts().head(3)

    # Convertir el resultado a un formato de lista de diccionarios
    result = [{"Puesto numero {} más recomendado es".format(i+1): juego} for i, juego in enumerate(top_games.index)]

    return result

# Ejemplo de uso con el año 2013
result_2013 = UsersRecommend(2013)
print(result_2013)

[{'Puesto numero 1 más recomendado es': 'Team Fortress 2'}, {'Puesto numero 2 más recomendado es': 'Rust'}, {'Puesto numero 3 más recomendado es': 'Counter-Strike: Global Offensive'}]


C:\Users\Transtermo\AppData\Local\Temp\ipykernel_8848\1235767909.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_reviews = df_reviews[


## Top tres menos recomendadas.

In [11]:
def UsersNotRecommend(year):
    # Convertir la columna 'release_date' a tipo datetime
    df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

    # Filtrar las reviews para el año dado con recomendación negativa y comentarios negativos
    filtered_reviews = df_reviews[
        (df_reviews['recommend'] == False) &
        (df_reviews['sentiment_analiysis'] == 0) &  # 0 para comentarios negativos
        (pd.to_datetime(df_games['release_date']).dt.year == year)
]
    # Unir con el DataFrame de juegos para obtener los nombres de los juegos
    merged_reviews = pd.merge(filtered_reviews, df_games[['id', 'app_name']], left_on='item_id', right_on='id')

    # Contar la cantidad de veces que aparece cada juego
    top_games = merged_reviews['app_name'].value_counts().head(3)

    # Convertir el resultado a un formato de lista de diccionarios
    result = [{"Puesto numero {} menos recomendado es".format(i+1): juego} for i, juego in enumerate(top_games.index)]

    return result

# Ejemplo de uso con el año 2013
result_2013 = UsersNotRecommend(2013)
print(result_2013)

[{'Puesto numero 1 menos recomendado es': 'Rust'}, {'Puesto numero 2 menos recomendado es': 'Miscreated'}, {'Puesto numero 3 menos recomendado es': 'Robocraft'}]


C:\Users\Transtermo\AppData\Local\Temp\ipykernel_8848\675684571.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_reviews = df_reviews[


## Analisis de sentimiento por año.

In [12]:
def sentiment_analysis(year):
    # Unir las reseñas con la información de los juegos
    merged_reviews = pd.merge(df_reviews, df_games[['id', 'release_date']], left_on='item_id', right_on='id')

    # Filtrar las reseñas para el año dado
    filtered_reviews = merged_reviews[
        (pd.to_datetime(merged_reviews['release_date'], errors='coerce').dt.year == year)
    ]

    # Contar la cantidad de registros por categoría de sentimiento
    sentiment_counts = filtered_reviews['sentiment_analiysis'].value_counts()

    # Crear el diccionario de resultados
    result = {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }

    return result

# Ejemplo de uso con el año 2013
result_2013 = sentiment_analysis(2000)
print(result_2013)

{'Negative': 4, 'Neutral': 57, 'Positive': 43}
